In [1]:
import numpy as np
import pandas as pd
from PIL import Image

from FloodImageClassifier import FloodImageClassifier

from sklearn.metrics import confusion_matrix, roc_auc_score

In [2]:
def confuse(y_true, y_pred):
    return pd.DataFrame(
        data = confusion_matrix(y_true, y_pred).T,
        columns = ['real false', 'real true'],
        index = ['pred false', 'pred true']
    )

In [3]:
def evl(y_true, y_pred, title):
    matrix = confusion_matrix(y_true, y_pred).T
    tp = matrix[1, 1]
    tn = matrix[0, 0]
    fp = matrix[1, 0]
    fn = matrix[0, 1]
    return pd.DataFrame(
        data = {
            'accuracy': [np.round((tp + tn) / (tp + tn + fp + fn), 3)],
            'sensitivity': [np.round(tp / (tp + fn), 3)],
            'specificity': [np.round(tn / (tn + fp), 3)],
            'precision': [np.round(tp / (tp + fp), 3)],
            'ROC AUC': [np.round(roc_auc_score(y_true, y_pred), 3)],
        },
        index = [title]
    )

### Testing class

In [4]:
fic = FloodImageClassifier(
    '../binary_images/flood_image_classifier_mlp_128_16.sav',
    '../binary_images/images/'
)

In [3]:
image_file_name = 'img_0.jpg'
img = Image.open(fic.path_to_images + image_file_name)

In [4]:
fic.predict_x(image_file_name)

False

In [5]:
df = fic.predict(pd.Series([f'img_{i}.jpg' for i in range(10)]))

In [6]:
df

0    False
1     True
2     True
3     True
4     True
5     True
6    False
7     True
8     True
9     True
dtype: bool

### Testing on train data

In [8]:
with open('../binary_images/data/non_flood_images.txt') as file:
    file_contents = file.readlines()

non_flood_images =[int(num.strip()) for num in (''.join(file_contents)).split(',')]

In [9]:
non_flood_images

[6,
 10,
 13,
 33,
 40,
 57,
 59,
 72,
 106,
 114,
 118,
 127,
 129,
 131,
 134,
 135,
 137,
 139,
 140,
 143,
 144,
 145,
 148,
 149,
 150,
 151,
 154,
 157,
 159,
 165,
 166,
 171,
 172,
 173,
 177,
 178,
 183,
 187,
 192,
 193,
 195,
 196,
 197,
 199,
 200,
 201,
 202,
 203,
 205,
 206,
 207,
 208,
 209,
 211,
 213,
 215,
 217,
 219,
 221,
 222,
 226,
 229,
 230,
 231,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 255,
 256,
 258,
 259,
 260,
 261,
 262,
 266,
 267,
 268,
 271,
 272,
 273,
 274,
 275,
 276,
 278,
 279,
 281,
 285,
 286,
 287,
 289,
 291,
 294,
 297,
 299,
 300,
 301,
 302,
 303,
 305,
 307,
 309,
 311,
 312,
 313,
 314,
 318,
 322,
 323,
 324,
 325,
 327,
 329,
 330,
 331,
 332,
 336,
 339,
 341,
 342,
 343,
 347,
 348,
 350,
 352,
 355,
 356,
 358,
 360,
 361,
 363,
 370,
 372,
 373,
 376,
 379,
 383,
 388,
 389,
 390,
 393,
 394,
 395,
 399,
 401,
 402,
 403,
 405,
 409,
 410,
 413,
 414,
 415,
 

In [10]:
len(non_flood_images)

365

In [12]:
y_true = [0 if i in non_flood_images else 1 for i in range(908)]

In [13]:
%%time
y_pred = fic.predict(pd.Series([f'img_{i}.jpg' for i in range(908)]))

CPU times: user 13min 32s, sys: 11 s, total: 13min 43s
Wall time: 3min 26s


In [19]:
confuse(y_true, y_pred)

,real false,real true
pred false,365,52
pred true,0,491


In [25]:
evl(y_true, y_pred, 'MLP')

,accuracy,sensitivity,specificity,precision,ROC AUC
MLP,0.943,0.904,1.0,1.0,0.952


### Testing on test data

In [14]:
fic = FloodImageClassifier(
    '../binary_images/flood_image_classifier_mlp_128_16.sav',
    '../binary_images/test_images/',
    threshold = 0.52 # nudge model toward specificity
)

In [15]:
with open('../binary_images/data/non_flood_test_images.txt') as file:
    file_contents = file.readlines()

non_flood_test_images =[int(num.strip()) for num in (''.join(file_contents)).split(',')]

In [16]:
y_true = [1 if i in non_flood_test_images else 0 for i in range(89)]

In [17]:
%%time
y_pred = fic.predict(pd.Series([f'img_test_{i}.jpg' for i in range(89)]))

CPU times: user 1min 23s, sys: 1.14 s, total: 1min 24s
Wall time: 21.2 s


In [18]:
confuse(y_true, y_pred)

,real false,real true
pred false,27,19
pred true,13,30


In [19]:
evl(y_true, y_pred, 'MLP')

,accuracy,sensitivity,specificity,precision,ROC AUC
MLP,0.64,0.612,0.675,0.698,0.644


In [20]:
false_positive_examples = []
false_negative_examples = []
for i in range(89):
    if y_true[i] and not y_pred[i]:
        false_positive_examples.append(i)
    elif not y_true[i] and y_pred[i]:
        false_negative_examples.append(i)

In [21]:
false_positive_examples

[3, 8, 11, 12, 14, 15, 18, 20, 24, 25, 34, 36, 42, 46, 49, 50, 70, 80, 88]

In [22]:
false_negative_examples

[4, 7, 17, 23, 35, 39, 54, 57, 64, 72, 73, 74, 82]